### Grammar of Graphics

Begrepet er fra Wilkinson (1999) og senere utviklet og popularisert av [Wickham (2008)](https://ggplot2-book.org/). Denne gramatikken lar oss tenke litt systematisk på sammenhengen mellom underliggende data og den grafiske representasjonen i figurer. Merk at vi også her skiller mellom syntaks og semantikk; det er fullt mulig å konstruere grafiske representasjoner som er grammatisk korrekte, men som er stygge eller meningsløse. 

Figuren inneholder ulike geometrisk objekter (sirkler, linjer, barer, mm). Det er en mapping mellom data og de ulike estetiske egenskapene til objektene (plassering, størrelse, form, farge). Vi kan representere data direkte, men i praksis vil vi gjøre transformasjoner som enten gir aggregerte mål (sum/antall) eller sammendragsmål (sentraltendens/spredning). Innholdet i mappingen er angitt ved en *scale*. Figuren må inneholde en *guide* som litt uformelt angir den inverse av scale. Vi konstruerer figuren ved å legge på geometriske objekter lagvis.

Jeg skal nå gi en *high-level* oversikt over hvordan grammatikken er implementert i ggplot

#### Geometrier

I praksis kunne vi hatt én *geomfunksjon* der vi spesifiserer hele mappingen mellom data og estetiske egenskaper til geometri, som kunne sett omtrent slik ut
```R
ggplot(data) +
    geom(object='bar', aes(x='col'), height='col_y', stat='count', width=.., ..)
```
Dette blir veldig verbost! Derfor er det i stedet en lang rekke ulike geomfunksjoner som har defaultverdier på omtrent alle egenskapene. Det eneste vi trenger å spesifiserer er estetisk mapping. Merk at hver geom korresponderer med en default statistisk transformasjon (og omvendt). Merk også at det ulike geomfunksjoner som bruker samme geometrisk objekt, men ulik statistisk transformasjon (eks barplot og histogram ?).

Vi kan sette egenskaper
1. Gjennom mapping fra data til estetikk, altså inni aes()
2. Gjennom å sette attributt utenfor aes(); har da konstant verdi (eg: farge) som ikke avhenger av data

Mye av arbeidet blir gjort gjennom geomfunksjoner, men ikke alt. Skal nå se på andre funksjoner som håndterer andre aspekter av grammatikken.

#### Posisjon

Vi må ha regel for hvordan vi skal håndtere overlapp mellom geometriske objekter i figuren, for eksempel at barer blir mappet til samme x-koordinat. Har fem ulike strategier
1. Jitter (legge på litt støy, aktuelt for scatterplot)
2. Dodge (plassere ved siden av hverandre)
3. Stack (plassere oppå hverandre)
4. Fill (plassere oppå hverandre og normalisere høyde slik at konstant)
5. Identitet (ikke gjøre noe)

#### Scale

Scale spesifiserer innholdet i mappingen fra *datarommet* til *estetikkrommet*. Utledning av scale består av tre steg
1. Transformasjon: f.eks. log(x)
2. Trening: må lære definisjonsmengden av mapping (min og maks for kontinuerlig, mulige verdier for diskret)
3. Mapping: hmm..

Skjønte ikke poenget der. Uansett, har i hovedsak tre typer scales som vi kan spesifisere
1. Position: scale_x_continious() ...
2. Farge: scale_colour()
3. Manuell: hmm...

Hvis jeg f.eks. vil ha log transformasjon av axis eller få kontroll over fargevalg så tror jeg at jeg gjør det med scale-funksjon..

##### Guide

Vi kan tenke på *guide* som samlebetgnelse på informasjon i figuren som angir det inverse av scale; hvordan vi mapper tilbake fra estetisk egenskap til data. Dette blir i stor grad håndtert automatisk og det finnes ikke noe eget axis-objekt eller legend-objekt der vi eksplisitt angir verdier.

#### Facet

Vi kan lage samme type plot på ulike subset av data. Dette er alternativ til å gruppere data med estetisk egenskap og er strategi for å håndtere overlapp. Gjør det enklere å se på trend innad i gruppe, men kan være vanskelig å sammenligne små forskjeller mellom grupper. Har to funksjoner
1. facet_grid: subset definert av to variabler, verdi ut fra plassering i 2d grid
2. facet_wrap subset definert av én variabel, 1d array lagt ut i 2d grid der plassering ikke har betydning

#### Theme

I ggplot skilles det mellom ting som er knyttet til grammatikk (mapping mellom data og grafikk) og semantikk (det som kun påvirker visuelle)... ikke helt presist..

Uansett, håndtere alt som har med tekst (font og størrelse), plassering av legend, bakgrunnsgrid og sånn gjennom theme.

Vil ha det stilrent og elegant! Kan bruke ferdiglagde themes som har konsistent stilvalg, eller så kan jeg fikle argumentene selv. Merk: default er veldig stygg!!!

### ggplot2

Skal nå se litt mer konkret på implementeringen

```R
ggplot(data = <DATA>) + 
    <GEOM_FUNCTION>(
        mapping = aes(<MAPPINGS>),
        stat = <STAT>, 
        position = <POSITION>
                   ) +
    <COORDINATE_FUNCTION> +
    <FACET_FUNCTION> +
    labs(
        title=, 
        x=,
        y=
        ) +
    xlim(min, max) +
    theme(
        param=param_value
         )
```

I praksis må vi bare spesifisere data, geom og mapping siden ggplot håndterer det andre under the hood.
Kan spesifisere global data og mapping i ggplot. Denne kan overkjøres av lokal som spesifiseres inni geom.
Det er også vesentlig poeng at vi kan legge geomer oppå hverandre.

Det er også poeng at ggplot tar ting fra dataframe.. Hvis vi vil endre på ting kan vi derfor enten endre dataframen eller gjøre endringer inne i ggplottet. Og veldig viktig at variablene har riktig datatype (factor/numerisk) for at plot skal fungere !

Noen ganger vil jeg konstruere data for plottet. Det er ikke noe problem å organisere data som dataframe for at det skal ha riktig format, bare gjør 
```R
tibble(x=sec) %>% mutate(y=f(x)) %>%  ggplot() 
```

Verdi fra andre kolonner. Dytter det inn i aes()
- group = 
- fill = 
- shape = 
- lty = 

Vi må overkjøre defaults lagsvis... 

```R
ggplot(data) +
    geom_(aes(x=,y=,lty=z,color=v)) +
    scale_color_() + # kan kjøre manualt, men har masse funksjoner
    scale_linetype_()
```

#### Geomer

##### Relasjoner

Visulisere relasjon mellom to kontinuerlige variabler:
- geom_point(, shape="open circle" eller shape=2)
- geom_smooth (linje fittet med lokal reg, LOESS. Kan spesifisere method="lm" hvis jeg vil ha lineær reg)


##### Fordelinger

- geom_histogram(aes(x=)
- geom_density(aes(x=), fill=)
- geom_boxplot(aes(x=discrete,y=kontinuerlig)) # kan eventuelt ha det horisontalt
- geom_violin(aes(x=discrete,y=kontinuerlig)) # gir litt mer detaljert beskrivelse av tetthet.


##### Kategorisk

- geom_bar(aes(x)) (antall med hver verdi av x) <- kan spesifisere position="fill" hvis vil vise andel
- geom_col(aes(x,y))  bruker count av y for hver x til å bestemme høyde
- geom_count(aes(x,y)) (tror den tar gjsnitt y for hver x men idk)


#### Posisjon

hm

#### Scale

#### Koordinat

- kan ønske å endre måleenhet på akse, _scale__x_continious(trans="..") eller scale_x_log10
- scale_x_continious(limits=,breaks=) # kan sette begge lik NULL for å ikke vise y akse

Andre scales:
- tror legend som angir farger, størrelse og sånt kalles for scales
- scale_size()
- scale_x_date()
- scales pakke som gir meg litt options for formatering i tekst (ie ticks, labels mm.), kan få prosent, dollartegn, spesifisere formatering med punktum i tall. ikke så lett å jobbe med strings i dette programmet.


#### Farge

Kan spesifisere indivduelle farger, men i praksis er det mest aktuelt å finne en fin palette
- scale_color_brewer()
- scale_color_gradientn(colours = rainbow(100)) # litt usikker på når dette er mer relevant


#### facet

Lage subplots med masse lingnede plots

- facet_wrap lager separat plot for hvert nivå av kategorisk z, facet_wrap(~z) 
- facet_grid lager separat hvor hver kombinasjon av (z,w), facet_grid(w\~z). Tenker z er kolonne og w er rekke. Hvis jeg bare vil ha én variabel så setter jeg facet_grid(.\~z)..


```R
ggplot() +
    geom_(mapping=aes()) +
    facet_grid(
               rows=,
               cols=,
               )
```

#### Tekst

- geom_text(x,y,label). Kan enten gjøre på spesifikk punkt med labed="blabla" eller bruke ting fra dataframe
- geom_text_repel() fra ggrepel gjør mye av arbeidet under the hood. gir meg automatiske offsets som ser bra ut


#### Theme

- theme() funksjon er ganske fleksibel for å velge ting som er spesifisert theme.
- theme(legend.position="top"),theme(legend.position="none") m.m
- theme(panel.grid = element_blank()) hvis jeg vil fjerne grid
- kan spesifisere disse tingene selv eller velge et premade theme, f.eks. laste inn fra pakken ggthemes

Bruker blant annet til å håndtere tekststørrelse og font på labels og sånn
```R
theme(text = element_text(size=20))
```


#### Annet

- geom_errorbar(x, ymin, ymax) gir meg barer så jeg kan vise konfidensintervall til punktestimat. må få disse inn i en dataframe først.

- geom_hline(yintercept)
- geom_vline(xintercept)
- ggsave(filename) # lagrer siste displayet plot som default
- fortify(mod) # ggplot bruker dataframe, bruker fortify() på fittet modell til å få relevant output (residual, yhat, mm) inn i dataframe slik at jeg kan bruke ggplot til å visualisere. 


### cowplot

### ggrepel

### ggforce

### Eksempler

```R
ggplot(murders, aes(population/10**6,total)) +
    geom_point(aes(color=region),size=3) +
    geom_smooth(method="lm", se=F, color="darkgrey", lty=2) +
    geom_text_repel(aes(label=abb)) +
    scale_x_log10() +
    scale_y_log10() + 
    scale_color_discrete(name="Region") +
    theme_economist() +
    xlab("Populations in millions (log scale)") + 
    ylab("Total number of murders (log scale)") +
    ggtitle("US Gun Murders in 2010")

```

```R
  ggplot(aes(x=wage_st,y=freq,fill=state)) +
    geom_col(position=position_dodge2(preserve = "single")) +
    scale_x_continuous(breaks=seq(4.25,5.55,by=.1))+
    scale_fill_discrete(name="",labels=c("Pennsylvania","New Jersey")) +
    xlab("Begynnerlønn i $") +
    ylab("")+
    ggtitle("Februar 1992") +
    theme_economist()
```